Data Engineering Assignment: Debt Collection ETL and Basic Analysis

Name : Siddharth Sonkavade


In [1]:
import pandas as pd
import sqlite3
import os

In [2]:
file_path = "/content/5k_borrowers_data.csv"

In [3]:
if not os.path.exists(file_path):
    raise FileNotFoundError(f"The file {file_path} does not exist.")

# Extract data from CSV
data = pd.read_csv(file_path)

In [4]:
# Print column names to diagnose the issue
print("Column names:", data.columns)

Column names: Index(['Name', 'Date of Birth', 'Gender', 'Marital Status', 'Phone Number',
       'Email Address', 'Mailing Address', 'Language Preference',
       'Geographical Location', 'Credit Score', 'Loan Type', 'Loan Amount',
       'Loan Term', 'Interest Rate', 'Loan Purpose', 'EMI', 'IP Address',
       'Geolocation', 'Repayment History', 'Days Left to Pay Current EMI',
       'Delayed Payment'],
      dtype='object')


# **Data Cleaning**

In [5]:
# Convert column names to a consistent format
data.columns = [col.lower().replace(' ', '_') for col in data.columns]

In [6]:
# Standardize date formats
data['date_of_birth'] = pd.to_datetime(data['date_of_birth'], format='%d-%m-%Y')

In [7]:
# Ensure numeric columns are in the correct format
numeric_columns = ['phone_number', 'credit_score', 'loan_amount', 'loan_term', 'interest_rate', 'emi']
for column in numeric_columns:
    data[column] = pd.to_numeric(data[column], errors='coerce')

In [8]:
# Clean phone numbers and email addresses
data['phone_number'] = data['phone_number'].astype(str).str.replace(r'\D', '', regex=True)
data['email_address'] = data['email_address'].str.strip().str.lower()

In [9]:
# Standardize categorical data
categorical_columns = ['gender', 'marital_status', 'language_preference', 'loan_purpose', 'delayed_payment']
for column in categorical_columns:
    data[column] = data[column].str.strip().str.lower().str.replace(' ', '_')

# **SQL Connection and ETL**

In [10]:
# Create a SQLite database connection
conn = sqlite3.connect('borrowers.db')

In [11]:
# Define the schema for the borrowers table
create_table_query = '''
CREATE TABLE IF NOT EXISTS borrowers (
    name TEXT,
    date_of_birth DATE,
    gender TEXT,
    marital_status TEXT,
    phone_number TEXT,
    email_address TEXT,
    mailing_address TEXT,
    language_preference TEXT,
    geographical_location TEXT,
    credit_score INTEGER,
    loan_amount REAL,
    loan_term INTEGER,
    interest_rate REAL,
    loan_purpose TEXT,
    emi REAL,
    ip_address TEXT,
    geolocation TEXT,
    repayment_history TEXT,
    days_left_to_pay_current_emi INTEGER,
    delayed_payment TEXT
)
'''

In [12]:
# Execute the query to create the table
conn.execute(create_table_query)
conn.commit()

In [13]:
# Load the data into the SQLite database
data.to_sql('borrowers', conn, if_exists='replace', index=False)

5000

In [14]:
# Verify that the data has been loaded correctly
loaded_data = pd.read_sql('SELECT * FROM borrowers', conn)
loaded_data.head()

,name,date_of_birth,gender,marital_status,phone_number,email_address,mailing_address,language_preference,geographical_location,credit_score,...,loan_amount,loan_term,interest_rate,loan_purpose,emi,ip_address,geolocation,repayment_history,days_left_to_pay_current_emi,delayed_payment
0,Khushi Balan,1986-02-19 00:00:00,male,single,2169182416,xbhakta@example.com,"29/74, Mander, Kulti 156906",marathi,Danapur,491,...,34398,15,0.16,medical_emergency,2545.36,10.65.217.95,"-49.3275015, 58.067192","[{'Payment Date': datetime.date(2023, 5, 3), '...",2,no
1,Umang Chatterjee,1980-08-13 00:00:00,male,married,4521712306,ivaidya@example.org,73/885\nSharma Marg\nSolapur 386449,malayalam,Bangalore,325,...,96856,9,0.20,home_renovation,11678.35,192.168.179.132,"14.951437, -136.491335","[{'Payment Date': datetime.date(2023, 5, 1), '...",4,yes
2,Adira Dara,1995-08-05 00:00:00,female,widowed,4615155004,loyalvihaan@example.net,H.No. 468\nRaval Zila\nNanded 490253,malayalam,Bijapur,624,...,91301,10,0.16,home_renovation,9812.94,10.166.34.134,"64.013123, 175.275587","[{'Payment Date': datetime.date(2023, 5, 7), '...",7,no
3,Anvi Saini,2001-11-16 00:00:00,male,married,914836846677,tgaba@example.org,"04/25, Mandal Path, Guntakal 305639",gujarati,Korba,346,...,78981,14,0.28,education_fees,6678.01,10.107.161.197,"-11.6130395, 37.595772","[{'Payment Date': datetime.date(2023, 5, 10), ...",7,yes
4,Kartik Kade,1979-07-09 00:00:00,male,widowed,910186397558,sanakaur@example.net,"55/13, Srivastava Path\nRaipur-801775",malayalam,Jorhat,321,...,89953,1,0.29,wedding_expenses,92126.86,192.168.255.28,"-71.7186905, 49.352990",[],11,no


#**Queries**

In [15]:
'''What is the average loan amount for borrowers who are more than 5 days past due? '''
query_a = '''
SELECT AVG(loan_amount) AS average_loan_amount
FROM borrowers
WHERE days_left_to_pay_current_emi > 5
'''

In [16]:
avg_loan_amount = pd.read_sql(query_a, conn).iloc[0]['average_loan_amount']
print("Average loan amount for borrowers who are more than 5 days past due:")
print(avg_loan_amount)

Average loan amount for borrowers who are more than 5 days past due:
54683.15681685196


In [17]:
'''Who are the top 10 borrowers with the highest outstanding balance? '''
query_b = '''
SELECT name, loan_amount - (emi * loan_term) AS outstanding_balance
FROM borrowers
ORDER BY outstanding_balance DESC
LIMIT 10
'''

In [18]:
top_borrowers = pd.read_sql(query_b, conn)
print("\nTop 10 borrowers with the highest outstanding balance:")
print(top_borrowers)


Top 10 borrowers with the highest outstanding balance:
              name  outstanding_balance
0  Jayant Bhandari              -108.01
1   Riaan Bhargava              -122.31
2      Prisha Char              -149.79
3        Ivan Keer              -152.70
4        Ira Varty              -156.64
5      Riaan Lalla              -158.91
6        Urvi Virk              -161.40
7     Yasmin Divan              -163.69
8     Kiara Kurian              -166.93
9      Bhavin Bala              -168.79


In [19]:
'''List of all borrowers with good repayment history'''
query_c = '''
SELECT *
FROM borrowers
WHERE delayed_payment = 'no'
ORDER BY credit_score DESC
LIMIT 10
'''

In [20]:
good_repayment_history_borrowers = pd.read_sql(query_c, conn)
print("\nList of all borrowers with good repayment history:")
print(good_repayment_history_borrowers)


List of all borrowers with good repayment history:
             name        date_of_birth  gender marital_status  phone_number  \
0    Shamik Joshi  1958-06-09 00:00:00  female       divorced  911093729717   
1       Shalv Lad  1990-07-13 00:00:00  female         single    4589119073   
2       Anvi Saha  1963-04-28 00:00:00  female        married    5437600486   
3    Kaira Doctor  1993-06-07 00:00:00  female         single    8270610885   
4     Aarna Salvi  2006-02-02 00:00:00  female       divorced  917866622488   
5      Aaina Dhar  1976-10-03 00:00:00    male        married  912339472539   
6     Zaina Kunda  2002-10-01 00:00:00    male       divorced     331501984   
7  Keya Jayaraman  1984-09-12 00:00:00  female        married  910090460649   
8     Adira Magar  1961-08-02 00:00:00  female        widowed    3044785047   
9     Divit Sahni  2003-12-12 00:00:00    male         single    4517399636   

                email_address                               mailing_address  \

In [21]:
'''Brief analysis wrt loan type'''
query_d = '''
SELECT loan_purpose, COUNT(*) AS num_borrowers, AVG(loan_amount) AS avg_loan_amount
FROM borrowers
GROUP BY loan_purpose
'''

In [22]:
loan_type_analysis = pd.read_sql(query_d, conn)
print("\nBrief analysis with respect to loan type:")
print(loan_type_analysis)



Brief analysis with respect to loan type:
         loan_purpose  num_borrowers  avg_loan_amount
0  debt_consolidation           1020     55704.978431
1      education_fees           1053     56063.981956
2     home_renovation            956     54320.202929
3   medical_emergency            981     54787.685015
4    wedding_expenses            990     55490.034343


In [23]:
conn.close()

#**Report Writing**

In [28]:
report_content = f"""
Analysis Report

a. Average loan amount for borrowers who are more than 5 days past due:
--------------------------------------------------------------------------------
{avg_loan_amount:.2f}

b. Top 10 borrowers with the highest outstanding balance:
--------------------------------------------------------------------------------
{top_borrowers[['name', 'outstanding_balance']].to_string(index=False)}

c. List of all borrowers with good repayment history:
--------------------------------------------------------------------------------
{good_repayment_history_borrowers['name'].to_string(index=False)}

d. Brief analysis with respect to loan type:
--------------------------------------------------------------------------------
{loan_type_analysis.to_string(index=False)}
"""


In [29]:
report_file_path = 'borrowers_analysis_report.txt'
with open(report_file_path, 'w') as file:
    file.write(report_content)

In [30]:
report_file_path

'borrowers_analysis_report.txt'